In [1]:
!git clone https://github.com/idealo/image-super-resolution.git

Cloning into 'image-super-resolution'...
remote: Enumerating objects: 1206, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 1206 (delta 0), reused 2 (delta 0), pack-reused 1203
Receiving objects: 100% (1206/1206), 14.98 MiB | 12.86 MiB/s, done.
Resolving deltas: 100% (583/583), done.
Filtering content: 100% (4/4), 152.87 MiB | 30.20 MiB/s, done.


In [3]:
cd image-super-resolution

/content/image-super-resolution


In [4]:
!python setup.py install

/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'description-file' will not be supported in future
        versions. Please use the underscore name 'description_file' instead.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
        ********************************************************************************

!!
  opt = self.warn_dash_deprecation(opt, section)
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
 

In [5]:
from ISR.models import RRDN

rrdn = RRDN(weights='gans')

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer RandomUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


17462488/17462488 [==============================] - 2s 0us/step


In [6]:
cd ..

/content


In [7]:
import cv2

In [8]:
!pip install facenet_pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 56.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 66.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [9]:
from facenet_pytorch import MTCNN

In [10]:
import torch

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


In [12]:
mtcnn = MTCNN(keep_all=True, device=device)

In [13]:
def preprocess_image(image, target_size=(200, 200)):
    # Normalize pixel values to range [0, 1]
    image = tf.cast(image, tf.float32) / 255.0

    # Resize the image
    image = tf.image.resize(image, target_size)

    # Convert to grayscale
    # image = tf.image.rgb_to_grayscale(image)

    # Remove blurriness and noise using Gaussian blur
    image = tf.image.random_brightness(image, max_delta=0.5)  # Adjust brightness
    image = tf.image.random_contrast(image, lower=0.2, upper=1.8)  # Adjust contrast

    return image

In [14]:
import tensorflow as tf
# Setup base model and freeze its layers (this will extract features)
base_model = tf.keras.applications.efficientnet.EfficientNetB7(include_top=False)
base_model.trainable = False

258076736/258076736 [==============================] - 2s 0us/step


In [15]:
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Dropout, Input, Dense, Activation, BatchNormalization, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Model

input_size = (200, 200, 3)
inputs = Input(shape=input_size)
X = base_model(inputs,training=False)
X = Conv2D(64, (3, 3), activation='relu', kernel_initializer=glorot_uniform(seed=0))(inputs)
X = BatchNormalization(axis=3)(X)
X = MaxPooling2D((3, 3))(X)

X = Conv2D(128, (3, 3), activation='relu')(X)
X = MaxPooling2D((2, 2), strides=(2, 2))(X)

X = Conv2D(256, (3, 3), activation='relu')(X)
X = MaxPooling2D((2, 2))(X)

X = Flatten()(X)

dense_1 = Dense(256, activation='relu')(X)
dense_2 = Dense(256, activation='relu')(X)
dense_3 = Dense(128, activation='relu')(dense_2)
dropout_1 = Dropout(0.4)(dense_1)
dropout_2 = Dropout(0.4)(dense_3)

# Gender output
output_gender = Dense(1, activation='sigmoid', name='gender_output')(dropout_1)

# Age output
output_age = Dense(1, activation='linear', name='age_output')(dropout_2)

model = Model(inputs=[inputs], outputs=[output_gender, output_age])


In [16]:
model.load_weights("/content/drive/MyDrive/model_bucket_weights.h5")

In [17]:

# video_capture = cv2.VideoCapture("/kaggle/input/zsvvsvzs/WhatsApp Video 2024-03-26 at 15.09.04_74f1850a.mp4")
# fps = int(video_capture.get(cv2.CAP_PROP_FPS))
# frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
# frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

# fourcc = cv2.VideoWriter_fourcc(*'XVID')

# out = cv2.VideoWriter("/kaggle/working/output_10.avi", fourcc, fps, (frame_width, frame_height))

In [18]:
def assign_age(age):
    if int(age) <= 12:
        return int(age) * 6 - 3
    elif int(age) == 13:
        return 77
    elif int(age) == 14:
        return 85
    elif int(age) == 15:
        return 95
    elif int(age) == 16:
        return 110
    else:
        return 115  # If age is beyond the specified buckets

In [19]:
# !pip install --upgrade opencv-python


In [20]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from PIL import Image
# Initialize MTCNN for face detection
mtcnn = MTCNN(keep_all=True)

# Load pre-trained FaceNet model
resnet = InceptionResnetV1(pretrained='casia-webface').eval()

  0%|          | 0.00/111M [00:00<?, ?B/s]

# AgePrediction from Video

In [21]:
import cv2
import numpy as np

# Initialize video capture
video_capture = cv2.VideoCapture("/content/vid_1.mp4")
fps = int(video_capture.get(cv2.CAP_PROP_FPS))
frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter("output_1.avi", fourcc, fps, (frame_width, frame_height))

# Dictionary to store age predictions for each tracked face
tracked_faces = {}

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Convert frame to RGB (if needed)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Detect faces in the frame
    boxes, _ = mtcnn.detect(frame_rgb)

    if boxes is not None:
        for face in boxes:
            x, y, w, h = face
            # Extract face region
            face_region = frame_rgb[int(y):int(y + h), int(x):int(x + w)]

            # Super resolution on face region
            enhanced_face_region = rrdn.predict(face_region)
            if(tf.size(enhanced_face_region)):
              img = preprocess_image(enhanced_face_region)
              # Predict age
              preds = model.predict(tf.expand_dims(img, axis=0))
              age = assign_age(preds[1].reshape(-1))
              gender = preds[0].reshape(-1)
              gender_pred = "Male" if gender <= 0.5 else "Female"

              # Find similar faces
              similar_face_found = False
              for tracked_face_box, tracked_age in tracked_faces.items():
                  if np.linalg.norm(np.array(face)[:2] - np.array(tracked_face_box)[:2]) < 20:
                      # Update age prediction for similar face
                      age = tracked_age
                      similar_face_found = True
                      break

              # If similar face is not found, add the current face to tracked faces
              if not similar_face_found:
                  tracked_faces[(x, y, w, h)] = age

              # Draw rectangle around the face
              # cv2.rectangle(frame, (int(x), int(y)), (int(x+w), int(y+h)), (0, 255, 0), thickness=2)

              # Add age prediction to the frame
              cv2.putText(frame, f"Age: {age}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
              cv2.putText(frame, f"Gender: {gender_pred}", (int(x), int(y) - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)

    # Write frame to output video
    out.write(cv2.cvtColor(frame, cv2.COLOR_RGB2BGR))

# Release video capture and video writer
video_capture.release()
out.release()
cv2.destroyAllWindows()


1/1 [==============================] - 3s 3s/step


<ipython-input-18-bfece6747b32>:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if int(age) <= 12:
<ipython-input-18-bfece6747b32>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(age) * 6 - 3


1/1 [==============================] - 0s 19ms/step


<ipython-input-18-bfece6747b32>:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  elif int(age) == 13:


1/1 [==============================] - 0s 18ms/step


<ipython-input-18-bfece6747b32>:6: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  elif int(age) == 14:


1/1 [==============================] - 0s 70ms/step


In [27]:
import pandas as pd
list_of_lists = []
for key, value in tracked_faces.items():
    list_of_lists.append([key, value])
df = pd.DataFrame(list_of_lists, columns=["face", "age"])

In [29]:
df.head()

,face,age
0,"(110.3046875, 238.7779541015625, 214.559494018...",57
1,"(224.82965087890625, 287.5671081542969, 264.07...",39
2,"(86.05589294433594, 173.0099639892578, 139.131...",51
3,"(67.56095123291016, 246.5062255859375, 116.124...",27
4,"(95.53399658203125, 278.0880432128906, 122.590...",21


In [30]:
df.to_csv("prediction.csv")

# Age Prediction from Image

In [35]:
img_file = input("Enter the image filename: ")
img_filename, extension = img_file.split('.')
output_img_filename = f"{img_filename}_output.{extension}"
img = cv2.imread(img_file)
tracked_faces = {}
frame = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
boxes, _ = mtcnn.detect(frame_rgb)

if boxes is not None:
    for face in boxes:
        x, y, w, h = face
        # Extract face region
        face_region = frame_rgb[int(y):int(y + h), int(x):int(x + w)]
        # Super resolution on face region
        enhanced_face_region = rrdn.predict(face_region)
        if(tf.size(enhanced_face_region)):
            img = preprocess_image(enhanced_face_region)
            # Predict age
            preds = model.predict(tf.expand_dims(img, axis=0))
            age = assign_age(preds[1].reshape(-1))
            gender = preds[0].reshape(-1)
            gender_pred = "Male" if gender <= 0.5 else "Female"

            # Find similar faces
            similar_face_found = False
            for tracked_face_box, tracked_age in tracked_faces.items():
                if np.linalg.norm(np.array(face)[:2] - np.array(tracked_face_box)[:2]) < 20:
                    # Update age prediction for similar face
                    age = tracked_age
                    similar_face_found = True
                    break

            # If similar face is not found, add the current face to tracked faces
            if not similar_face_found:
                tracked_faces[(x, y, w, h)] = age

            # Draw rectangle around the face
            # cv2.rectangle(frame, (int(x), int(y)), (int(x+w), int(y+h)), (0, 255, 0), thickness=2)

            # Add age prediction to the frame
            cv2.putText(frame, f"Age: {age}", (int(x), int(y) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
            cv2.putText(frame, f"Gender: {gender_pred}", (int(x), int(y) - 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36, 255, 12), 2)
            cv2.imwrite(output_img_filename, frame)

Enter the image filename: img_1.jpg
1/1 [==============================] - 0s 19ms/step


<ipython-input-18-bfece6747b32>:2: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  if int(age) <= 12:
<ipython-input-18-bfece6747b32>:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return int(age) * 6 - 3


In [36]:
list_of_lists = []
for key, value in tracked_faces.items():
    list_of_lists.append([key, value])
df = pd.DataFrame(list_of_lists, columns=["face", "age"])

In [38]:
df.to_csv("image_prediction_1.csv", index =False)